# Charting a path to cinematic success
#### Milestone 2 - TheArmadaOfBadassEmpanadas

In this notebook, we embark on a journey to craft a data pre-processing pipeline, essential for the success of our project. Our exploration centers around the CMU Movie Corpus dataset, with a keen focus on developing two key components: a plot summary data frame and a movie metadata data frame.

Guided by the narrative of our project, we aim to unearth initial insights through a meticulous process of data cleaning, reformating, and enrichment:

- **Data Cleaning:** This involves identifying and rectifying inaccuracies or inconsistencies in our dataset. It includes tasks like handling missing values, removing duplicates, and correcting data entry errors. This process ensures the reliability and quality of our data for accurate analysis.

- **Reformatting into Database-Like Structure:** We will restructure the data into a more organized, database-like format. This step includes categorizing data into tables, establishing relational links, and indexing for efficient querying and retrieval. This structured approach enhances data accessibility and usability.

- **Fetching Information from Wikipedia API:** To enrich our dataset, we will integrate external data by fetching relevant information from the Wikipedia API. 

After we have cleaned, reformatted, and enhanced our data, we will explore various dimensions like box office revenues, representation of women in film, and sentiment analysis. Our exploration aims to identify correlations among different variables and glean meaningful insights. Ultimately, this process is designed to address the key questions outlined in our project's storytelling narrative, enabling us to draw significant conclusions from the data.

In [2]:
# Imports
#--- General imports
import pandas as pd
import json
import os
from tqdm import tqdm
from collections import Counter
import random

#--- Imports for data vizualization
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from wordcloud import WordCloud

#--- Imports for data cleaning
import pywikibot
import requests
import datetime

#--- Imports for queries on Wikidata
from SPARQLWrapper import SPARQLWrapper, JSON

#--- Imports for summary extraction
import gzip
import shutil
import xml.etree.ElementTree as ET
import stanza

In [3]:
# Data source: http://www.cs.cmu.edu/~ark/personas/

# Define the paths
DATA_PATH = 'data/'
CLEAN_DATA_PATH = 'clean_data/'

## Step 1: Data cleaning 

The objective here is to clean and standardize the dataset. This involves removing JSON strings and unifying the data format. Additionally, countries in the dataset will be categorized into global regions for later analysis.

In [3]:
# As we already cleaned the data before submission, we can skip those steps
PERFORM_CLEANING = False

### 1.1 Grouping the countries into world regions

Initially, the plan was to classify movies into Hollywood, Bollywood, and Nollywood categories. However, due to a lack of sufficient Nollywood data and the absence of this categorization in the dataset, this approach was modified.

In [4]:
# Load the movies data
movies_df = pd.read_csv(DATA_PATH + 'movie.metadata.tsv', delimiter='\t', header=None, names=['wiki_id', 'id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres'])

# Convert the json string to a list of countries
movies_df["countries"] = movies_df["countries"].apply(lambda x: list(json.loads(x).values()))

In [5]:
# Count the number of movies that are assigned to each countries. 
# Duplicate the movies if they have multiple countries.

nb_movies_per_country = movies_df["countries"].explode().value_counts()
display(nb_movies_per_country.head(20))

countries
United States of America    34408
India                        8411
United Kingdom               7868
France                       4395
Italy                        3163
Japan                        2647
Canada                       2534
Germany                      2393
Argentina                    1468
Hong Kong                    1240
Spain                        1136
Australia                    1114
South Korea                   887
Mexico                        870
Netherlands                   840
Sweden                        657
West Germany                  647
China                         645
Denmark                       610
Soviet Union                  564
Name: count, dtype: int64

None of the 20 first most represented country can be used for Nollywood. Since there also will be data cleaning in the next steps due to NaN information. We decided to not use this categorization.

Instead, movies will be grouped based on the following global regions:
- North America: United States of America, Canada
- Europe: United Kingdom, France, Italy, Germany, Spain, and others
- Asia: Japan, Hong Kong, China, South Korea, Taiwan
- India: India, Pakistan

In [6]:
areas = {
	"Northen America": set(["United States of America", "Canada"]),
	"Europe": set(["United Kingdom", "France", "Italy", "Germany", "Spain", "West Germany", "Belgium", "German Democratic Republic", "Ireland", "Switzerland", "Austria", "England", "Luxembourg", "Portugal"]),
	"Asia": set(["Japan", "Hong Kong", "China", "South Korea", "Taiwan"]),
	"India": set(["India", "Pakistan"])
}
country_to_area = {country: area for area, countries in areas.items() for country in countries}

def get_area(country):
	if country in country_to_area:
		return country_to_area[country]
	else:
		return "Other"

# Count the number of movies that are assigned to each area.
# Duplicate the movies if they have multiple countries.
movies_areas_df = movies_df.copy()
movies_areas_df = movies_areas_df['countries'].apply(lambda x: [get_area(country) for country in x])
nb_movies_per_area = movies_areas_df.explode().value_counts()
display(nb_movies_per_area)

countries
Northen America    36942
Europe             21571
Other              13731
India               8600
Asia                5595
Name: count, dtype: int64

As we can see, those four categories will have enough data for us to still be able to clean them and do the analysis.

Keep in mind that those categories are not perfect, and that some countries are missing, or overlap. That's why we now will group the countries into those categories.

First, one thing that stroke us is that each movie can have multiple countries. The question is how to deal with that ? In fact most of the movies have not much countries, but some have a lot.

In [ ]:
# Plot the bar plot of the number of countries per movie (log scale)
plt.figure(figsize=(12, 8))
plt.title("Number of countries per movie")
plt.xlabel("Number of countries")
plt.ylabel("Number of movies")
sns.countplot(x=movies_df["countries"].apply(len))
plt.yscale('log')
plt.show()

Another thing that we saw, is that 'United States of America' are frequently part of the countries of a movie. 
One way to see it is to plot a heatmap of the countries of a movie.

In [ ]:
movies_copy_df = movies_df.copy()
# Only keep the countries that are in the areas
movies_copy_df["countries"] = movies_copy_df["countries"].apply(lambda x: [country for country in x if country in country_to_area])
# Only keep the movies that have at least one country
movies_copy_df = movies_copy_df[movies_copy_df["countries"].apply(len) > 0]
# Only keep the movies that have countries that are not in the same area
movies_copy_df["areas"] = movies_copy_df["countries"].apply(lambda x: set([country_to_area[country] for country in x]))
movies_copy_df = movies_copy_df[movies_copy_df["areas"].apply(lambda x: len(x) > 1)]

In [ ]:
# Display a heatmap of the combinations of countries
countries_df = pd.DataFrame(columns=country_to_area.keys(), index=country_to_area.keys())
countries_df = countries_df.fillna(0)
for index, row in movies_copy_df.iterrows():
	countries_list = list(row["countries"])
	pairs = combinations(countries_list, 2)
	for pair in pairs:
		countries_df[pair[0]][pair[1]] += 1
		countries_df[pair[1]][pair[0]] += 1

plt.figure(figsize=(15, 15))
sns.heatmap(countries_df, annot=True, cmap="YlGnBu", fmt='g', cbar_kws={'label': 'Count'})
plt.show()

In [ ]:
# Display a heatmap of the combination of areas
areas_df = pd.DataFrame(columns=areas.keys(), index=areas.keys())
areas_df = areas_df.fillna(0)
for index, row in movies_copy_df.iterrows():
	areas_list = list(row["areas"])
	pairs = list(combinations(areas_list, 2))
	for pair in pairs:
		areas_df[pair[0]][pair[1]] += 1
		areas_df[pair[1]][pair[0]] += 1

sns.heatmap(areas_df, annot=True, cmap="YlGnBu", fmt='g', cbar_kws={'label': 'Count'})
plt.show()

TODO: What can we conclude about these heatmaps?

Since nothing is specified in the dataset about what represent those countries, we will assume that we cannot choose when they are not in the same region.

One way to handle this problem is to say that if there is only one country in the list, we will use it. If there are multiple countries and every of those countries are in the same world region, we will group them in the region directly. If there are multiple countries and they are not all in the same region, we will use the Wikipedia API to get the real country of the movie.

In fact, this third option involving GET requests takes a lot of time (less than 2 movies per second), so we wanted to avoid using it as much as possible.

In [ ]:
# Set default area to None
movies_df["area"] = None

print(f"There are {len(movies_df)} movies in total")

In [ ]:
# Set the areas if the number of countries played in is 1
movies_without_area_df = movies_df[movies_df["countries"].apply(lambda x: len(x) == 1)]
movies_df.loc[movies_without_area_df.index, "area"] = movies_without_area_df["countries"].apply(lambda x: get_area(x[0]))

print(f"Fetching areas for {len(movies_without_area_df)} movies (remaining {len(movies_df[movies_df['area'].isnull()])})")
display(movies_df.loc[movies_without_area_df.index, ["id", "countries", "area"]].head(10))

In [ ]:
def get_area_from_unanimity(countries):
	area = None
	for country in countries:
		if country in country_to_area:
			if area is None:
				area = country_to_area[country]
			elif area != country_to_area[country]:
				return None
		else:
			return None
	return area

# For each remaining movie (only the ones with None as area), if the countries played in are in the same area, set the area to that area
movies_without_area_df = movies_df[movies_df["area"].isnull()]
movies_df.loc[movies_without_area_df.index, "area"] = movies_without_area_df["countries"].apply(lambda x: get_area_from_unanimity(x))

print(f"Combining areas for {len(movies_without_area_df)} movies (remaining {len(movies_df[movies_df['area'].isnull()])})")
display(movies_df.loc[movies_without_area_df.index, ["id", "countries", "area"]].head(10))

As the last step took us more than 8 hours to run, we decided to save the data in a csv file, and to load it directly in the next steps.

In [ ]:
wikipedia = pywikibot.Site('en', 'wikipedia')
country_property = 'P495'

def get_area_from_wikipedia(movie_wiki_id):
	try:
		wiki_api_url = f'https://en.wikipedia.org/w/api.php?action=query&pageids={movie_wiki_id}&format=json'
		json_data = requests.get(wiki_api_url).json()
		movie_title = json_data["query"]["pages"][str(movie_wiki_id)]["title"]
		movie_page = pywikibot.Page(wikipedia, movie_title)
		movie_item = pywikibot.ItemPage.fromPage(movie_page)
		movie_item_dict = movie_item.get()
		movie_country = movie_item_dict["claims"][country_property][0].getTarget().labels["en"]
		return country_to_area[movie_country] if movie_country in country_to_area else "Other"
	except:
		return "Not found"
	
# For each remaining movie (that had the number of countries played in != 1), we set the area retrieved from wikipedia by scraping
movies_without_area_df = movies_df[movies_df["area"].isnull()]
if PERFORM_CLEANING:
	movies_df.loc[movies_without_area_df.index, "area"] = movies_without_area_df["wiki_id"].apply(lambda x: get_area_from_wikipedia(x))

print(f"Retriving areas for {len(movies_without_area_df)} movies (remaining {len(movies_df[movies_df['area'].isnull()])})")
display(movies_df.loc[movies_without_area_df.index, ["id", "countries", "area"]].head(10))

Now, the most part of the movies has a country and therefore a world region attached to it. It will then be used during the rest of the analysis.

### 1.2 Cleaning and reformat the metadata

We will now clean the data and reformat it. We will do the following steps:
- Remove the json strings
- Add a column that only contains the year of the movie
- Separate the multiple values into other dataframes (to have a database like structure)
- Save them in a csv file

In [ ]:
# Separate the json columns for the movies table

# Separate the languages to another table
movies_df["languages"] = movies_df["languages"].apply(lambda x: list(json.loads(x).values())) # Convert the json string to a list of languages
movies_languages_df = movies_df[["id", "languages"]].explode("languages").reset_index(drop=True)
movies_languages_df = movies_languages_df.rename(columns={"languages": "language", "id": "movie_id"})
display(movies_languages_df.head())

# Separate the genres to another table
movies_df["genres"] = movies_df["genres"].apply(lambda x: list(json.loads(x).values())) # Convert the json string to a list of genres
movies_genres_df = movies_df[["id", "genres"]].explode("genres").reset_index(drop=True)
movies_genres_df = movies_genres_df.rename(columns={"genres": "genre", "id": "movie_id"})
display(movies_genres_df.head())

# Separate the countries to another table
# The countries are already a list of countries
movies_countries_playedin_df = movies_df[["id", "countries"]].explode("countries").reset_index(drop=True)
movies_countries_playedin_df = movies_countries_playedin_df.rename(columns={"countries": "country", "id": "movie_id"})
display(movies_countries_playedin_df.head())

# Drop the useless columns (that have been separated)
movies_df = movies_df.drop(columns=["languages", "genres", "countries"])
display(movies_df.head())

In [ ]:
# Get year from formats 'YYYY-MM-DD' or 'YYYY'
def get_year(date: str):
	try:
		year = datetime.datetime.strptime(date, '%Y-%m-%d').year
		return year
	except:
		year = date
		return year

# Add the column release_year to the movies table
movies_df["release_year"] = movies_df["release_date"].apply(lambda x: get_year(x))
display(movies_df.head())

In [ ]:
# Store the summaries of the movies in a separate table, using the movie_id instead of the wiki_id
# We could have directly merged the summaries with the movies table, but it is a very large text field

# Load the summaries
movies_summaries_df = pd.read_csv(DATA_PATH + 'plot_summaries.txt', delimiter='\t', header=None, names=['wiki_id', 'summary'])
display(movies_summaries_df.head())

# Merge the summaries with the movies table, to change the wiki_id to movie_id
movies_summaries_df = movies_summaries_df.merge(movies_df[["wiki_id", "id"]], on="wiki_id")
movies_summaries_df = movies_summaries_df.drop(columns=["wiki_id"])
movies_summaries_df = movies_summaries_df.rename(columns={"id": "movie_id"})
display(movies_summaries_df.head())

In [ ]:
# Store the characters in a separate table

# Load the characters
characters_df = pd.read_csv(DATA_PATH + 'character.metadata.tsv', delimiter='\t', header=None, names=['movie_wiki_id', 'movie_id', 'movie_release_date', 'name', 'actor_birth_date', 'actor_gender', 'actor_height','actor_ethinicity_id', 'actor_name', 'actor_age', 'actor_map_id', 'id', 'actor_id'])
# Add the column release_year to the character table
characters_df["movie_release_year"] = characters_df["movie_release_date"].apply(lambda x: get_year(x))
display(characters_df.head())

In [ ]:
# Clean the characters categories table

# Load the characters categories
characters_categories_df = pd.read_csv(DATA_PATH + 'tvtropes.clusters.txt', delimiter='\t', header=None, names=['category', 'meta'])
display(characters_categories_df.head())

# Fetch the characters ids from the meta column, and remove the other information that are redundant with the characters table
characters_categories_df["meta"] = characters_categories_df["meta"].apply(lambda x: json.loads(x))
characters_categories_df["character_id"] = characters_categories_df["meta"].apply(lambda x: x["id"])
characters_categories_df = characters_categories_df.drop(columns=["meta"])
display(characters_categories_df.head())

It is now time to save everything in a csv file, so that we can load it in the next steps.

In [ ]:
if PERFORM_CLEANING:
	# Make sure to create the clean_data folder before running this script
	
	# Remove all the previous files from the clean_data folder
	for filename in os.listdir(CLEAN_DATA_PATH):
		os.remove(CLEAN_DATA_PATH + filename)

	# Save the data
	movies_df.to_csv(CLEAN_DATA_PATH + 'movies.csv', index=False)
	movies_languages_df.to_csv(CLEAN_DATA_PATH + 'movies_languages.csv', index=False)
	movies_genres_df.to_csv(CLEAN_DATA_PATH + 'movies_genres.csv', index=False)
	movies_countries_playedin_df.to_csv(CLEAN_DATA_PATH + 'movies_countries_playedin.csv', index=False)
	movies_summaries_df.to_csv(CLEAN_DATA_PATH + 'movies_summaries.csv', index=False)
	characters_df.to_csv(CLEAN_DATA_PATH + 'characters.csv', index=False)
	characters_categories_df.to_csv(CLEAN_DATA_PATH + 'characters_categories.csv', index=False)

To see what we did, here is a little overview of the tables related to the movies:
- movies.csv
- movies_countries_playedin.csv
- movies_genres.csv
- movies_languages.csv
- movies_summaries.csv

To use all of those tables (except movies), we will need to merge them to the movie table (in a database like structure).

In [ ]:
movies_df = pd.read_csv(CLEAN_DATA_PATH + 'movies.csv')
display(movies_df.head())

# Add the language to the movies table
languages_df = pd.read_csv(CLEAN_DATA_PATH + 'movies_languages.csv')
movies_df = pd.merge(movies_df, languages_df, left_on="id", right_on="movie_id", how="left")
display(movies_df.head())

In this example, the languages can be multiple for a single movie, so the joined table will have multiple rows for a single movie.

### 1.3. Plot summaries extraction and cleaning
The corenlp_plot_summaries dataset is composed of gzipped folders, each containing a single xml file with matching name. Each folder-file corresponds to one movie, and the name is the movie ID.

We thus need to extract all zipped folder, and get the XML file. To do this, we will follow these steps:
1. Extract the gzipped folder using gzip package.
2. Move extracted files to a unique folder in clean_data.
3. Remove the empty folders after moving the files.

Same as above, this step only need to be performed one time.

In [ ]:
# Path to the zipped folders
NLP_DATA_PATH = '../corenlp_plot_summaries/'

# Path to the new data folder
NLP_SUMMARIES = 'clean_data/NLP_summaries/'

# Create the new data folder if it doesn't exist
if not os.path.exists(NLP_SUMMARIES):
    os.makedirs(NLP_SUMMARIES)

# Iterate through each gzipped folder
for root, dirs, files in os.walk(NLP_DATA_PATH):
    for file in files:
        # Check if the file is a gzipped file
        if file.endswith('.gz'):
            # Path to the gzipped file contained in the folder
            gz_path = os.path.join(root, file)

            # Create a folder name based on the gzipped file name, removing the extension
            folder_name = os.path.splitext(file)[0]

            # Path to the destination folder
            destination_folder = os.path.join(NLP_SUMMARIES, folder_name)

            # Unzip the contents of the gzipped file to the destination folder
            with gzip.open(gz_path, 'rb') as gz_file:
                with open(destination_folder, 'wb') as out_file:
                    shutil.copyfileobj(gz_file, out_file)

# Move all XML files from the unzipped folders to the clean data folder
for root, dirs, files in os.walk(NLP_SUMMARIES):
    for file in files:
        # Check if the file is an XML file
        if file.endswith('.xml'):
            # Path to the XML file
            xml_path = os.path.join(root, file)

            # Move the XML file
            shutil.move(xml_path, os.path.join(NLP_SUMMARIES, file))

# Remove the empty folders left after moving the XML files
for root, dirs, files in os.walk(NLP_SUMMARIES, topdown=False):
    for folder in dirs:
        folder_path = os.path.join(root, folder)
        os.rmdir(folder_path)

Now, each XML file is stored in clean_data folder and can be read to analyze summary informations.

## Step 2: Data exploration and visualization


The groundwork is now in place for easy data accessibility, given our planned analysis. It's time to dive deeper into our data. Our initial focus is dual: evaluating data integrity, particularly regarding missing data, and gaining an overview of its characteristics. We will sequentially assess the following  parameters—film box-office revenue, movie genre, actor gender, and actor ethnicity — than we plan to integrate within our data story. This stage serves as a critical precursor, guiding us to identify parameters suitable for exploration in the next milestone and determining the direction that our next analysis will take. 

#### 2.1 Exploration of the box-office revenue of movies

First, we need separate the data and create individuals dataframe from the movies dataframe for each geographical region previously defined, using the area feature added before.

In [4]:
#Load the data
movies_df = pd.read_csv(CLEAN_DATA_PATH + 'movies.csv')

#Create a separate dataframe with data of each area
movies_revenue_df=movies_df.copy()
asia_movies_df = movies_revenue_df[movies_revenue_df["area"] == "Asia"]
europe_movies_df = movies_revenue_df[movies_revenue_df["area"] == "Europe"]
india_movies_df = movies_revenue_df[movies_revenue_df["area"] == "India"]
northen_america_movies_df = movies_revenue_df[movies_revenue_df["area"] == "Northen America"]

display(asia_movies_df.head())
display(europe_movies_df.head())
display(india_movies_df.head())
display(northen_america_movies_df.head())

,wiki_id,id,name,release_date,revenue,runtime,area,release_year
11,24229100,/m/07kjkz6,Lady Snowblood 2: Love Song of Vengeance,1974,NaN,89.0,Asia,1974
43,26878691,/m/0f400r,Mysterious Island,1982,NaN,100.0,Asia,1982
56,13281430,/m/03c0d85,My Name is Fame,2006,NaN,94.0,Asia,2006
108,24196090,/m/07k8x7p,The World of Geisha,1973-11-03,NaN,73.0,Asia,1973
114,17780234,/m/0479c20,Banana Club,1996-04-13,NaN,NaN,Asia,1996


,wiki_id,id,name,release_date,revenue,runtime,area,release_year
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,Europe,1987
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,Europe,1983
9,2345652,/m/075f66,City of the Dead,1960,NaN,76.0,Europe,1960
13,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,Europe,1989
16,30388930,/m/0g5qvzg,1919,1984,NaN,99.0,Europe,1984


,wiki_id,id,name,release_date,revenue,runtime,area,release_year
27,20604092,/m/0523t_1,Anbu Thozhi,2007-08-17,NaN,NaN,India,2007
30,11448183,/m/02rc_h4,Bindiya Chamkegi,1984-01-20,NaN,NaN,India,1984
31,9633533,/m/02pml15,Vandanam,1989,NaN,168.0,India,1989
35,29528534,/m/04j0lfk,Anokha Rishta,1986,NaN,180.0,India,1986
37,29772142,/m/0fq260_,Karayilekku Oru Kadal Dooram,2010-12-31,NaN,110.0,India,2010


,wiki_id,id,name,release_date,revenue,runtime,area,release_year
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,Northen America,2001
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,Northen America,2000
5,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,Northen America,1913
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,Northen America,1938
10,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,Northen America,1930


Before performing any analysis on this feature, we need to check if there are some missing box-office revenue data. If so, we need to determine to which extent, in order to decide the best way to handle it, and determine how it could impact our analysis.

In [ ]:
# compute the proportion of missing revenue data
def percentage_missing_data(df, region, ax):
    # Indicator variable showing that something is NA
    df.loc[:,["revenue_is_na"]] = df.revenue.isna()

    # % of missing data
    missing_revenue_data = df.groupby("release_year").revenue_is_na.mean()
    
    # Plot it
    ax.plot(missing_revenue_data)
    ax.set(xlabel='Year', ylabel='Percentage')
    ax.set_title(region)
    x_values = ax.get_xticks()
    new_x = [x_values[i] for i in [0, len(x_values) //2, -1]]
    ax.set_xticks(new_x)
    
    #return overall percentage of missing data
    prop_missing_data=sum(df["revenue_is_na"])*100/len(df)
    return prop_missing_data

fig, axs = plt.subplots(4, sharey=True, figsize=(15, 10), layout='constrained')
fig.suptitle("Percentage of missing data per year")
prop_missing_asia=percentage_missing_data(asia_movies_df, "Asia", axs[0])
prop_missing_europe=percentage_missing_data(europe_movies_df, "Europe", axs[1])
prop_missing_india=percentage_missing_data(india_movies_df, "India", axs[2])
prop_missing_northen_america=percentage_missing_data(northen_america_movies_df, "Northen America", axs[3])
plt.show()

# PrintOverall proportion of missing data
print("Percentage of missing revenue data:")
print("- Asia: %.2f" % prop_missing_asia)
print("- Europe: %.2f" % prop_missing_europe)
print("- India: %.2f" % prop_missing_india)
print("- Northen America: %.2f" % prop_missing_northen_america)

In general, the data exhibits a lot of missing values, ranging from 82.19% to 99.67% across various regions. Particularly noteworthy is the near 100% absence of revenue data for India, as well as substantial gaps over extended timeframes in the other regions. Consequently, conducting a meaningful analysis becomes unfeasible. The dataset lacks uniformity in revenue representation among areas, and represents only select years. As a result, we've opted against analyzing film revenue and instead shifted focus to other parameters that offer richer insights.

#### 2.2 Exploration of the movie genres

In [ ]:
# Load the data
genres_df = pd.read_csv(CLEAN_DATA_PATH + 'movies_genres.csv')
display(genres_df.head())

#Merge movie genre to movie dataset
movies_genres_df = pd.merge(movies_df, genres_df, left_on="id", right_on="movie_id", how="left")
display(movies_genres_df.head())

Similarly to the movie revenue, we separate the data by creating individuals dataframe from the movies dataframe for each geographical region previously defined.

In [ ]:
asia_genres_df = movies_genres_df[movies_genres_df["area"] == "Asia"]
display(asia_genres_df.head())
europe_genres_df = movies_genres_df[movies_genres_df["area"] == "Europe"]
display(europe_genres_df.head())
india_genres_df = movies_genres_df[movies_genres_df["area"] == "India"]
display(india_genres_df.head())
northen_america_genres_df = movies_genres_df[movies_genres_df["area"] == "Northen America"]
display(northen_america_genres_df.head())

As for the revenue, before performing any analysis on this caracteristic, we need to determine the proportion of missing data.

In [ ]:
#Function to compute the proportion of missing data about movie genre
def percentage_missing_data(df, region, ax):
    # Indicator variable showing that something is NA
    df.loc[:,["genre_is_na"]] = df.genre.isna()

    # % of missing data
    missing_genre_data = df.groupby("release_year").genre_is_na.mean()
    
    # Plot it
    ax.plot(missing_genre_data)
    ax.set(xlabel='Year', ylabel='Percentage')
    ax.set_title(region)
    x_values = ax.get_xticks()
    new_x = [x_values[i] for i in [0, len(x_values) //2, -1]]
    ax.set_xticks(new_x)
    
    #Compute overall percentage of missing data
    tot_missing_data=sum(df["genre_is_na"])*100/len(df)
    return tot_missing_data

fig, axs = plt.subplots(4, sharey=True, figsize=(15, 10), layout='constrained')
fig.suptitle("Percentage of missing data per year")
prop_tot_asia = percentage_missing_data(asia_genres_df, "Asia", axs[0])
prop_tot_europe = percentage_missing_data(europe_genres_df, "Europe", axs[1])
prop_tot_india = percentage_missing_data(india_genres_df, "India", axs[2])
prop_tot_northen_america = percentage_missing_data(northen_america_genres_df, "Northen america", axs[3])
plt.show()

print("Percentage of missing data for genre:")
print("- Asia: %.2f" % prop_tot_asia)
print("- Europe: %.2f" % prop_tot_europe)
print("- India: %.2f" % prop_tot_india)
print("- Northen America: %.2f" % prop_tot_northen_america)

The absence of data for movie genres is notably minimal, varying between 0.17% to 5.73%. Indian -the only area surpassing  1%- display a slightly higher absence at 5.73%, yet it remains within reasonable bounds. Moreover, the missing data appear evenly spread across the years. Consequently, we've chosen to eliminate these movies from the subsequent phase of genre analysis.

In [ ]:
#Get rid of movies whose genre is not specified
print("Removing films from which the genre is not specified:")
print("- Number of Asian movies before:", len(asia_genres_df))
asia_genres_df=asia_genres_df[~asia_genres_df["genre_is_na"]]
print("- Number of Asian movies after:", len(asia_genres_df))

print("- Number of European movies before:", len(europe_genres_df))
europe_genres_df=europe_genres_df[~europe_genres_df["genre_is_na"]]
print("- Number of European movies after:", len(europe_genres_df))

print("- Number of Indian movies before:", len(india_genres_df))
india_genres_df=india_genres_df[~india_genres_df["genre_is_na"]]
print("- Number of Indian movies after:", len(india_genres_df))

print("- Number of Northen American movies before:", len(northen_america_genres_df))
northen_america_genres_df=northen_america_genres_df[~northen_america_genres_df["genre_is_na"]]
print("- Number of Northen American movies after:", len(northen_america_genres_df))

With missing data addressed, it's time to have an overview of diverse movie genres across various geographical areas! First, let's have a look at the number of genres in each area, and the quantity of movies belonging to each of them.

In [ ]:
# For each region, group the movies by their genre, and count the number of movies per genre
def group_by_genre(genre_df):
    genre_count = genre_df.explode('genre')
    genre_count = genre_df.groupby('genre').count()['id'].reset_index()
    genre_count = genre_count.rename(columns={'id': 'movies_count'})
    genre_count = genre_count.sort_values('movies_count', ascending=False)
    display(genre_count.head(10))
    return genre_count
    
asia_genre_count=group_by_genre(asia_genres_df)
europe_genre_count=group_by_genre(europe_genres_df)
india_genre_count=group_by_genre(india_genres_df)
northen_america_genre_count=group_by_genre(northen_america_genres_df)

#Print the number of different genres
print("Number of genre categories for Asian film: ", len(asia_genre_count))
print("Number of genre categories for European film: ", len(europe_genre_count))
print("Number of genre categories for Indian film: ", len(india_genre_count))
print("Number of genre categories for Northen American film: ", len(northen_america_genre_count))

There's quite a variety of genres here! To streamline our analysis, we'll focus on the primary movie genres in each geographical region. We'll retain genres that appear in at least 2% of the films for this purpose, and group all the other in a category named "Others".

In [ ]:
#Keep only genre that represents 2% of the movie, and group the other in a categorie name "others"
def add_category_other(genre_count, threshold):
    genre_count_df = genre_count[genre_count["movies_count"]/genre_count["movies_count"].sum()>threshold]
    new_row = pd.DataFrame(data = {'genre': ['Others'], 'movies_count' : [genre_count[genre_count["movies_count"]/genre_count["movies_count"].sum()<=threshold].movies_count.sum()]})
    genre_count_df = pd.concat([genre_count_df, new_row])
    return genre_count_df

asia_genre_count_df = add_category_other(asia_genre_count, 0.02)
europe_genre_count_df = add_category_other(europe_genre_count, 0.02)
india_genre_count_df = add_category_other(india_genre_count, 0.02)
northen_america_genre_count_df = add_category_other(northen_america_genre_count, 0.02)

fig, ax = plt.subplots(2, 2, figsize=(15, 15))

# Dictionary to store genre-color mapping
genre_colors = {}

# Loop through each region and plot the pie chart with consistent colors
for i, (region_df, title) in enumerate(zip([asia_genre_count_df, europe_genre_count_df, india_genre_count_df, northen_america_genre_count_df],['Asia', 'Europe', 'India', 'North America'])):
    colors = []
    for genre in region_df['genre']:
        if genre not in genre_colors:  # If the genre doesn't have a color assigned yet, generate a random color and save it
            genre_colors[genre] = sns.color_palette('hls', n_colors=len(region_df['genre']))[len(colors)]
        colors.append(genre_colors[genre])  # Use the saved color for the genre
    row = i // 2
    col = i % 2
    ax[row][col].pie(region_df.movies_count, labels=region_df.genre, autopct='%1.1f%%', colors=colors)
    ax[row][col].set_title(title)
fig.suptitle('Proportion of movie genre per region')
plt.show()

We observe prevalent genres shared across geographical boundaries, like drama consistently securing a top-three position in every region. However, certain genres exhibit more regional specificity, such as martial arts in Asia or musicals in India. Investigating these similarities and differences will take center stage in the upcoming project milestone as we delve deeper into our analysis. Particularly intriguing is examining the primary genres shared among countries and exploring each genre's uniqueness regarding ethnicity and gender representation.

It's essential to highlight that the "other" category regroup a significant portion of the movies. This arises from the high specificity of keywords used for genre classification, resulting in numerous, smaller genre categories, many of which contain a low number of movies. Thus, we should consider a potential grouping of related genres into broader categories. Hence, data analysis approach for genres during next milestone will likely involve employing clustering techniques, leveraging pre-trained word embeddings to combine genres into more encompassing categories.

#### 2.3 Gender representation

#### 2.4 Ethnicity

In this exploration of ethnicity representation, we are going to use the file characters.csv to analysis the actors' ethnicity in movies produced in the different geograhic areas.

In [ ]:
# Load the data and display the head of the created Dataframe
characters_df = pd.read_csv(CLEAN_DATA_PATH + 'characters.csv')
display(characters_df.head())

The main goal of our project is to analyse the film industry across four key geographical regions. To do so, our initial step involves generating four "sub-dataframes" to examine character data, enabling us to explore ethnicities in diverse global regions.

In [ ]:
# Load movies data set in order to retreive ID-area corresponding
movie_df = pd.read_csv(CLEAN_DATA_PATH + 'movies.csv')
merged_df = pd.merge(characters_df, movie_df[['id', 'area']], left_on='movie_id', right_on='id', how='left', suffixes=('', '_drop'))
merged_df = merged_df.drop('id_drop', axis=1)

In [ ]:
# Creat four sub-dataframe extracted from character.csv for the four regions
asia_characters_df = merged_df[merged_df['area'] == 'Asia']
europe_characters_df = merged_df[merged_df['area'] == 'Europe']
india_characters_df = merged_df[merged_df['area'] == 'India']
northen_america_characters_df = merged_df[merged_df['area'] == 'Northen America']

**Check for missing data in actor ethnicity**  
The fisrt information needed for this exploration is the proportion of missing values for the actors' ethnicity.

In [ ]:
# compute the proportion of missing ethnicity data
def percentage_missing_data(df, region, ax):
    # Indicator variable showing that something is NA
    df.loc[:,["ethnicity_is_na"]] = df.actor_ethinicity_id.isna()

    # % of missing data
    missing_ethnicity_data = df.groupby("movie_release_year").ethnicity_is_na.mean()
    
    # Plot it
    ax.plot(missing_ethnicity_data)
    ax.set(xlabel='Year', ylabel='Percentage')
    ax.set_title(region)
    x_values = ax.get_xticks()
    new_x = [x_values[i] for i in [0, len(x_values) //2, -1]]
    ax.set_xticks(new_x)
    
    #return overall percentage of missing data
    prop_missing_data=sum(df["ethnicity_is_na"])*100/len(df)
    return prop_missing_data

fig, axs = plt.subplots(4, sharey=True, figsize=(15, 10), layout='constrained')
fig.suptitle("Percentage of missing data per year")
prop_missing_asia=percentage_missing_data(asia_characters_df.copy(), "Asia", axs[0])
prop_missing_europe=percentage_missing_data(europe_characters_df.copy(), "Europe", axs[1])
prop_missing_india=percentage_missing_data(india_characters_df.copy(), "India", axs[2])
prop_missing_northen_america=percentage_missing_data(northen_america_characters_df.copy(), "Northen America", axs[3])
plt.show()

# PrintOverall proportion of missing data
print("Percentage of missing revenue data:")
print("- Asia: %.2f" % prop_missing_asia)
print("- Europe: %.2f" % prop_missing_europe)
print("- India: %.2f" % prop_missing_india)
print("- Northen America: %.2f" % prop_missing_northen_america)

We can observe a significant portion of NaN values in actor ethnicity across our four distinct groups. We can notice that this percentage is slightly decreasing over the last decades, but it remains quit important and not negligeable in our final analysis conclusions. Due to resource constraints, we opt to remove these NaN values, as obtaining corresponding ethnicity for such a large number of actors through scraping is not feasible for us.

In [ ]:
len_asia_df_na = len(asia_characters_df)
asia_characters_df = asia_characters_df.dropna(subset=['actor_ethinicity_id'])
print("We removed ", len_asia_df_na - len(asia_characters_df), " character of asian movies with no registered ethnicity ID")
print("It remains ", len(asia_characters_df), " character of asian movies.")

len_europe_df_na = len(europe_characters_df)
europe_characters_df = europe_characters_df.dropna(subset=['actor_ethinicity_id'])
print("We removed ", len_europe_df_na - len(europe_characters_df), " character of european movies with no registered ethnicity ID")
print("It remains ", len(europe_characters_df), " character of european movies.")

len_india_df_na = len(india_characters_df)
india_characters_df = india_characters_df.dropna(subset=['actor_ethinicity_id'])
print("We removed ", len_india_df_na - len(india_characters_df), " character of indian movies with no registered ethnicity ID")
print("It remains ", len(india_characters_df), " character of indian movies.")

len_northen_america_df_na = len(northen_america_characters_df)
northen_america_characters_df = northen_america_characters_df.dropna(subset=['actor_ethinicity_id'])
print("We removed ", len_northen_america_df_na - len(northen_america_characters_df), " character of northen american movies with no registered ethnicity ID")
print("It remains ", len(northen_america_characters_df), " character of northen american movies.")

**Transform author ethnicity ID**  
The author ethnicity of each actor is registered in the dataset by a Freebase ID. In order to obtain the corresponding labels we retrieve information from the Wikidata knowledge base using a query language.

In [ ]:
def get_ethnicity(freebase_id):
    endpoint_url = "https://query.wikidata.org/sparql"
    query = """
    SELECT ?item ?itemLabel WHERE {
      ?item wdt:P646 '""" + str(freebase_id) + """'.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        if results["results"]["bindings"]:
            label = results["results"]["bindings"][0]["itemLabel"]["value"]
            return label
        else:
            return "No label"
    except Exception as e:
        return np.nan

In [ ]:
# For the 4 groups remove all ethnicity ID duplicates to make the query fastest
asia_df_unique_ethnicity = asia_characters_df.copy()
asia_df_unique_ethnicity = asia_df_unique_ethnicity.drop_duplicates(subset='actor_ethinicity_id', keep='first')

europe_df_unique_ethnicity = europe_characters_df.copy()
europe_df_unique_ethnicity = europe_df_unique_ethnicity.drop_duplicates(subset='actor_ethinicity_id', keep='first')

india_df_unique_ethnicity = india_characters_df.copy()
india_df_unique_ethnicity = india_df_unique_ethnicity.drop_duplicates(subset='actor_ethinicity_id', keep='first')

northen_america_df_unique_ethnicity = northen_america_characters_df.copy()
northen_america_df_unique_ethnicity = northen_america_df_unique_ethnicity.drop_duplicates(subset='actor_ethinicity_id', keep='first')

# Creat a Data Frame with only unique ethnicity present in our four groups of interest
df_unique_ethnicity = pd.concat([asia_df_unique_ethnicity, europe_df_unique_ethnicity, india_df_unique_ethnicity, northen_america_df_unique_ethnicity], ignore_index=True)
df_unique_ethnicity = df_unique_ethnicity.drop_duplicates(subset='actor_ethinicity_id', keep='first')
df_unique_ethnicity = df_unique_ethnicity[['actor_ethinicity_id']]

print("Number of different actor ethnicithies represented in all our movies of interest is: ", len(df_unique_ethnicity))

In [ ]:
# Get ethnicity labels corresponding to the different IDs thanks to Wikidata
df_unique_ethnicity['ethnicity_Label'] = df_unique_ethnicity['actor_ethinicity_id'].apply(get_ethnicity)

# Save the ethnicity labels / IDs corresponding data into a CSV file
df_unique_ethnicity.to_csv(CLEAN_DATA_PATH + 'etnicity_labels.csv', index=False)

In [ ]:
# Load the CSV file with the ethnicity ID-label mapping
ethnicity_mapping = pd.read_csv(CLEAN_DATA_PATH + 'etnicity_labels.csv')

# Add a new column of our character Data Frame with a new column cprresponding to ethnicity label
asia_characters_df = pd.merge(asia_characters_df, ethnicity_mapping, how='left', on='actor_ethinicity_id')
europe_characters_df = pd.merge(europe_characters_df, ethnicity_mapping, how='left', on='actor_ethinicity_id')
india_characters_df = pd.merge(india_characters_df, ethnicity_mapping, how='left', on='actor_ethinicity_id')
northen_america_characters_df = pd.merge(northen_america_characters_df, ethnicity_mapping, how='left', on='actor_ethinicity_id')

Certain ethnicity IDs are absent from the Wikidata database, making it impossible to retrieve their corresponding labels. In our function, these ethnicity IDs are linked to the label "No label." We decided to eliminate these labels and treat them as missing values.

In [ ]:
# Remove values with unknow ethnicity labels
asia_characters_df = asia_characters_df[asia_characters_df['ethnicity_Label'] != "No label"]
europe_characters_df = europe_characters_df[europe_characters_df['ethnicity_Label'] != "No label"]
india_characters_df = india_characters_df[india_characters_df['ethnicity_Label'] != "No label"]
northen_america_characters_df = northen_america_characters_df[northen_america_characters_df['ethnicity_Label'] != "No label"]

**Ethnic diversity across our different regions**  
To get a better idea of which ethnicities are represented in our four different groups and to identify the predominant one we decided to visualize this information through four separate word clouds. In these word clouds, the size of each ethnicity name is proportional to its frequency of occurrence.

In [ ]:
# Transform all different ethnicities present in Asian films into a list
word_list_asia = asia_characters_df['ethnicity_Label'].tolist()
word_list_europe = europe_characters_df['ethnicity_Label'].tolist()
word_list_india = india_characters_df['ethnicity_Label'].tolist()
word_list_northen_america = northen_america_characters_df['ethnicity_Label'].tolist()

# Group lists to be able to iterate over a loop
word_lists = [word_list_asia, word_list_europe, word_list_india, word_list_northen_america]
# Group list to print word cloud title
region_str_list = ["Asian movies actor ethnicities",
                   "European movies actor ethnicities",
                   "Indian movies actor ethnicities",
                   "Northen american movies actor ethnicities"]
colormaps = ['Oranges', 'Blues', 'PuRd_r', 'YlGn_r']

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
# Iterate through the word lists and titles
for i, region_list in enumerate(word_lists):
    # Convert float values to strings
    word_list = [str(word) for word in region_list if not isinstance(word, float)] 
    # Count occurrences for the different ethnicities
    word_counts = Counter(word_list) 
    # Generate the word cloud with size proportional to occurrences
    wordcloud = WordCloud(width=400, height=200, background_color='white', colormap=colormaps[i]).generate_from_frequencies(word_counts)  
    # Plot the word cloud on the corresponding subplot
    ax = axes[i // 2, i % 2]
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')   
    # Set the title for the subplot
    ax.set_title(region_str_list[i])

plt.tight_layout()
plt.show()

We notice that the ethnicities linked to actors in this CMU movie dataset are highly specific, making it challenging to form broader subgroups that ethically group different ethnicities together. Indeed we will have to face two major problems for a deeper an more genral exploration of ethicity representation:  
- Presence of two many precise categories ungroupable
- Occurence of many ethnicithies with a very small frequencies which are part of the dataset and we have to use them to avoid biases and misrepresentation in the results  

Film industries in our four region of the world are large melting pot of actors coming from different horizon. In order to countinue our exploration we are going to focus on the raw count of different ethnicities present in the different "film regions".

In [ ]:
# Creat a list grouping ouf four datasets
dataframes = [asia_characters_df, europe_characters_df, india_characters_df, northen_america_characters_df]
# Get unique ethnicities for each group
unique_ethnicities = [df['ethnicity_Label'].unique() for df in dataframes]
# Count the number of unique ethnicities for each group
counts_per_group = [len(ethni) for ethni in unique_ethnicities]
# Create a DataFrame with counts and names
df_counts = pd.DataFrame({'Counts': counts_per_group, 'Names': ["Asia", "Europe", "India", "Northen American"]})
# Sort the DataFrame by counts in descending order
df_counts = df_counts.sort_values(by='Counts', ascending=False)

# Plots the different counts
fig, ax = plt.subplots(figsize=(8, 4))

# Plots the different counts
ax.bar(range(len(df_counts)), df_counts['Counts'], align='center', alpha=0.7)

# Set labels, axis and title
ax.set_xlabel('Regions of the world')
ax.set_ylabel('Count of unique ethnicities (all years combined)')
ax.set_title('Count of unique ethnicities across the geographical regions')
ax.set_xticks(range(len(df_counts)))
ax.set_xticklabels(df_counts['Names'])
plt.show()

If we only look a raw countings of the different ethnicities present in the four different geographical groups over all years, we can clearly see that more different ethnicies are represented in Northern American and Western European movies compared to Asian and Indian filmss.  

However, caution is necessary when interpreting this plot, as it represents raw counts without considering the total number of movies produced by each industry. It is obvious that an industry which produces a huge amount of films provides more opportunities for ethnic representation.  

In order to go further in our exploration we are going to compare how this ethnicity diversity evolves across years in our four group when taking into account the amout of produced movies. The large proportion of missing values, which is unevenly distributed among the groups, makes these plots less reliable.

In [ ]:
def preprocess_df(df, raw_count):
    df['movie_release_date'] = pd.to_numeric(df['movie_release_date'], errors='coerce')
    df = df[df['movie_release_date'].between(1970, 2013)]
    
    # Group by 'movie_release_date' and calculate the number of rows for each year
    year_counts = df.groupby('movie_release_date').size()
    
    # Group by 'movie_release_date' and calculate the weighted count of unique ethnicities
    unique_ethnicities = df.groupby('movie_release_date')['ethnicity_Label'].nunique()

    if not(raw_count):
        # Ensure the correct alignment of counts with the corresponding years
        weighted_ethnicities = weighted_ethnicities = pd.Series((unique_ethnicities.values / year_counts.values)*100,
                                          index=unique_ethnicities.index)
    elif raw_count:
        weighted_ethnicities = unique_ethnicities
    
    return weighted_ethnicities, year_counts

In [ ]:
# Preprocess each DataFrame
unique_asian_ethnicities, asian_count  = preprocess_df(asia_characters_df, False)
unique_european_ethnicities, european_count = preprocess_df(europe_characters_df, False)
unique_indian_ethnicities, indian_count = preprocess_df(india_characters_df, False)
unique_na_ethnicities, na_count = preprocess_df(northen_america_characters_df, False)

unique_asian_ethnicities_count, asian_count  = preprocess_df(asia_characters_df, True)
unique_european_ethnicities_count, european_count = preprocess_df(europe_characters_df, True)
unique_indian_ethnicities_count, indian_count = preprocess_df(india_characters_df, True)
unique_na_ethnicities_count, na_count = preprocess_df(northen_america_characters_df, True)

# Create a common date range
year_range = range(1942, 2014)

# Reindex each DataFrame to fill NaN for missing years
unique_asian_ethnicities = unique_asian_ethnicities.reindex(year_range)
unique_european_ethnicities = unique_european_ethnicities.reindex(year_range)
unique_indian_ethnicities = unique_indian_ethnicities.reindex(year_range)
unique_na_ethnicities = unique_na_ethnicities.reindex(year_range)

unique_asian_ethnicities_count = unique_asian_ethnicities_count.reindex(year_range)
unique_european_ethnicities_count = unique_european_ethnicities_count.reindex(year_range)
unique_indian_ethnicities_count = unique_indian_ethnicities_count.reindex(year_range)
unique_na_ethnicities_count = unique_na_ethnicities_count.reindex(year_range)


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# Plotting
ax1.plot(asian_count.index, asian_count.values, marker='o', linestyle='-', color='tab:orange', markersize=3, label='asian movies')
ax1.plot(european_count.index, european_count.values, marker='o', linestyle='-', color='tab:blue', markersize=3, label='european movies')
ax1.plot(indian_count.index, indian_count.values, marker='o', linestyle='-', color='tab:pink', markersize=3, label='indian movies')
ax1.plot(na_count.index, na_count.values, marker='o', linestyle='-', color='tab:olive', markersize=3, label='northen american movies')
# Set labels and title
ax1.set_xlabel('Year of movie release')
ax1.set_ylabel('Count of released movies')
ax1.set_title('Count of released movies over the years')
ax1.legend(loc="upper left")
ax1.grid()

# Plotting
ax2.plot(unique_asian_ethnicities_count.index, unique_asian_ethnicities_count.values, marker='o', linestyle='-', color='tab:orange', markersize=3, label='asian movies')
ax2.plot(unique_asian_ethnicities_count.index, unique_european_ethnicities_count.values, marker='o', linestyle='-', color='tab:blue', markersize=3, label='european movies')
ax2.plot(unique_asian_ethnicities_count.index, unique_indian_ethnicities_count.values, marker='o', linestyle='-', color='tab:pink', markersize=3, label='indian movies')
ax2.plot(unique_asian_ethnicities_count.index, unique_na_ethnicities_count.values, marker='o', linestyle='-', color='tab:olive', markersize=3, label='northen american movies')
# Set labels and title
ax2.set_xlabel('Year of movie release')
ax2.set_ylabel('Raw count different actor ethnicities')
ax2.set_title('Raw count of different ethnicities over the years')
ax2.legend(loc="upper left")
ax2.grid()

# Plotting
ax3.plot(unique_asian_ethnicities.index, unique_asian_ethnicities.values, marker='o', linestyle='-', color='tab:orange', markersize=3, label='asian movies')
ax3.plot(unique_asian_ethnicities.index, unique_european_ethnicities.values, marker='o', linestyle='-', color='tab:blue', markersize=3, label='european movies')
ax3.plot(unique_asian_ethnicities.index, unique_indian_ethnicities.values, marker='o', linestyle='-', color='tab:pink', markersize=3, label='indian movies')
ax3.plot(unique_asian_ethnicities.index, unique_na_ethnicities.values, marker='o', linestyle='-', color='tab:olive', markersize=3, label='northen american movies')
# Set labels and title
ax3.set_xlabel('Year of movie release')
ax3.set_ylabel('Percentage of different actor ethnicities (%)')
ax3.set_title('Percentage of different ethnicities over the years')
ax3.legend(loc="upper left")
ax3.grid()

plt.tight_layout()
plt.show()

In order to simplify our interpretation, we decided to focus our attention on films released after 1970 due to a significant proportion of missing values and a limited number of available films before this date.

In the initial plot, we observe the film production trends across our four geographical regions:  
- The number of eastern asian films remains consistently low and quite constant over the years. This will make the ethnicity diversity proportional analysis more complicated for this region.
- Since the 1980s, film production has increased in the other three regions, offering more opportunities for ethnic representation diversification.
- European and Indian curves appear almost superimposed, facilitating a more straightforward comparison of ethnic representation between these two regions.
- The Northern American film industry appears significantly larger than the other three, contributing to the higher raw count in the previous bar plot. Even if the overall shape is quite similar to the one of European and Indian movies, the counting values are largely superior. 
- Last few years are associated with a drastic reduced amount of film available in our data set making comparisons difficult since 2010.

In the second plot, we observe the raw count of different ethnicities represented over the years:   
- The ethnicity count number increased the most in Northern American and European films.
- In Eastern Asian and Indian films we can also observed a raw count increase even if the different across years is smaller.

The third plot illustrates the percentage of different ethnicities represented in each region for different years. To achieve this, we divided the ethnicity count by the total count of films in a specific region and year:  
- Values for Eastern Asian unique ethnicities percentage are really non readable and comparable as we have a very few movies available. We observe a high variability of our curve over years.  
- Globaly the other three curves show a slight decreasing pattern. This suggests that, despite an increase in the number of films produced, ethnic diversity representation did not rise sufficiently to maintain a constant percentage or show an increase. Indeed if we compare the slope of curves of the first plot and the ones for the second (assuming the same y-axis for both), the second plot's slopes are notably smaller. This implies that the number of movies produced increased more rapidly than the diversification of ethnicities represented on screen.
- As we said before european and asian curve can be easily comparable as their number of produced films are quite similar. On this second plot, these two curves evolves in parallel, but the percentage of different ethnicities in  european movies remains larger than the one for asian movies. This indicates that, for an equivalent number of produced films, ethnic diversity is more significant in European-produced movies.

#### 2.5 Exploration of plot summaries

In this part, we will first analyze the results of the CoreNLP pipeline already run on the summaries, stored in ```clean_data/NLP_summaries```, and then apply sentiment analysis on the raw summaries

**CoreNLP results for one summary**

To begin, we will select one movie by specifying its ID and look at the data from the corresponding XML file.

In [ ]:
# Path to the new data folder
NLP_SUMMARIES = 'clean_data/NLP_summaries/'

# Select a movie
movie_id = 3217

# Path to the XML file
summary_test_file = NLP_SUMMARIES + str(movie_id) + '.xml'
print(summary_test_file)

Each `<token>` element represents a word from a sentence, identified by a unique `id`. Inside each `<token>` element, there are several child elements that provide more information about the token:

- `<word>`: The actual word in the text.
- `<lemma>`: The base or dictionary form of the word.
- `<CharacterOffsetBegin>` and `<CharacterOffsetEnd>`: The start and end positions of the word in the original text.
- `<POS>`: The part-of-speech tag for the word. POS tagging is the task of labeling the words in a sentence with their appropriate part of speech (noun, verb, adjective, etc.).
- `<NER>`: Named Entity Recognition tag. NER is a subtask of information extraction that seeks to locate and classify named entities in text into predefined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

The `<sentence>` element contains a set of these tokens, representing a sentence in the text. The `id` attribute of the `<sentence>` element indicates the order of the sentence in the document.

In [ ]:
# Parse the XML file and get the root element
tree = ET.parse(summary_test_file)
root = tree.getroot()

# Create an empty dict to store data
tokens_dict = {}

# Iterate over all 'token' elements in the document
for token in root.iter('token'):
    # Extract data from the child elements of 'token'
    word = token.find('word').text
    lemma = token.find('lemma').text
    pos = token.find('POS').text
    ner = token.find('NER').text
    
    # Store in dict
    token_id = token.attrib['id']
    tokens_dict[token_id] = {
        'word': word,
        'lemma': lemma,
        'POS': pos,
        'NER': ner,
    }

    # Print the extracted data
    #print(f'Word: {word}, Lemma: {lemma}, POS: {pos}, NER: {ner}')

# Print the dict
print(tokens_dict)

Each sentence also has dependency informations between tokens:

- `<basic-dependencies>`: These represent grammatical relationships between words in a sentence. For example, the dependencies between noun and verb.

- `<collapsed-dependencies>`: These are a simplified form of dependencies where certain types of indirect dependencies are collapsed into direct dependencies for easier processing.

In [ ]:
# Create an empty dict to store data
dep_dict = {}

# Extract and print dependencies
for i,dep in enumerate(root.iter('dep')):
    # Extract data
    dep_type = dep.attrib['type']
    governor = dep.find('governor').text
    dependent = dep.find('dependent').text

    # Store in dict
    dep_dict[i] = {
        'type': dep_type,
        'governor': governor,
        'dependent': dependent
    }
    
    # Print the extracted data
    # print(f"Dependency type: {dep_type}, Governor: {governor}, Dependent: {dependent}")

# Print the dict
print(dep_dict)

Finaly, dependencies between different sentences can be analyzed:

- `<coreferences>`: These represent instances where multiple expressions in a text refer to the same entity. 

In the XML, each `<coreference>` element represents a group of mentions that all refer to the same entity in the text. Each `<mention>` element within a `<coreference>` represents a specific instance where that entity is mentioned in the text.
The `<mention representative="true">` is the primary mention in the text that other mentions refer back to. 

The `<sentence>` tag within a `<mention>` indicates the sentence number where the mention occurs. The `<start>` and `<end>` tags indicate the position of the start and end of the mention within that sentence. The `<head>` tag indicates the head word of the mention.

To know what a coreference corresponds to in the actual text, we would need to find the sentences and word positions indicated by the `<sentence>`, `<start>`, and `<end>` tags in the text itself.

In [ ]:
# Create an empty dict to store data
coref_dict = {}

# Extract and print coreferences
for i,coref in enumerate(root.iter('coreference')):
    coref_dict[i] = {}
    for j,mention in enumerate(coref.iter('mention')):
        # Extract data
        sentence = mention.find('sentence').text
        start = mention.find('start').text
        end = mention.find('end').text

        # Store in dict
        coref_dict[i][j] = {
            'sentence': sentence,
            'start': start,
            'end': end
        }

        # Print the extracted data
        #print(f"Sentence: {sentence}, Start: {start}, End: {end}")

# Print the dict
print(coref_dict)

We finally get for each summary:
- A `token dictionnary` containing the word, lemma, POS and NER of each tokens (words) of the summary.
- A `dependency dictionnary` containing each gramatical dependencies found in the summary along with the dependency type.
- A `coreference dictionnary` referencing for each coreference (entity refered to) all the group of words related to it in a specific sentence.

The nexts steps of this analysis will consist in extracting such data for all the summaries and include the relevant informations for the different analysis axes above. This will be done for P3.

**Sentiment analysis for a few summaries**

In this part, we will apply sentiment analysis on the sentences of one summary. To do this, we use the stanza package

First, we will load the raw summaries from the cleaned data.

In [ ]:
# Define data path
DATA_PATH = 'clean_data/'

# Load summaries
summaries_df = pd.read_csv(DATA_PATH + 'movies_summaries.csv')

# Display dataframe
display(summaries_df)

Then, we will import the stanza model for sentiment analysis.

In [ ]:
# Download english model
stanza.download('en')

# initialize English neural pipeline
nlp = stanza.Pipeline(lang='en')

Now, we need to convert the summaries to stanza documents to be able to use the package:

In [ ]:
# Convert the summaries column of df into a list
summaries = summaries_df["summary"].tolist()

# Select a subset of summaries for testing
summaries_test = summaries[0:4]

# Wrap each document with a stanza.Document object
summaries_list = [stanza.Document([], text=p) for p in summaries_test]

We will now iterate on the test list to get the sentiment scores. The sentiment is expressed by a simple classification: 
- 1 = Negative sentiment
- 2 = Neutral sentiment
- 3 = Positive sentiment

Each sentence of the summaries is associated with such class, resulting in a list of sentiment scores for each summary.

We iterate on a loop instead of passing the list directly to the function to avoid running out of RAM on such large data.

In [ ]:
for i,summary in enumerate(summaries_list):
  print("---------")
  print(f"Summary {i+1}:")
  # Print the summary
  print(summaries_test[i])

  # Run nlp pipeline
  doc = nlp(summary)

  # Print sentences sentiments
  sentiment_scores = []
  for i, sentence in enumerate(doc.sentences):
    sentiment_scores.append(sentence.sentiment)
  print("Sentiment scores: ", sentiment_scores)

We thus get for each summary a list of sentiment scores corresponding to each sentence.